In [2]:
#! /usr/bin/python3.5

In [3]:
a=["A","B","C","D","hh"]

In [4]:
def chunkIt(seq, num):
   
    avg = len(seq) / float(num)
    out = []
    last = 0.0
    if num > 1:
        while last < len(seq):
            out.append(seq[int(last):int(last + avg)])
            last += avg
    else:
        return seq

    return out

In [5]:
def model(a_list,i):
    from datetime import datetime
    start_time = datetime.now()
    z=1000000000
    c=0
    if i !=0:
        for x in range(z/i):
            c= x*2
    else:
        for x in range(z):
            c=x*2
        
            
    print("list model",a_list,i)
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))


    return c

In [6]:
#model(a,0)

In [7]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
    #return [x.name for x in local_device_protos]

In [8]:
import subprocess, re, os, sys #https://github.com/yaroslavvb/stuff/blob/master/notebook_util.py
def run_command(cmd):
    """Run command, return output as string."""
    
    output = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True).communicate()[0]
    return output.decode("ascii")


In [9]:
def list_available_gpus():
    """Returns list of available GPU ids."""
    
    output = run_command("nvidia-smi -L")
    # lines of the form GPU 0: TITAN X
    gpu_regex = re.compile(r"GPU (?P<gpu_id>\d+):")
    result = []
    for line in output.strip().split("\n"):
        m = gpu_regex.match(line)
        assert m, "Couldnt parse "+line
        result.append(int(m.group("gpu_id")))
    return result

In [33]:
def gpu_memory_map(gpu_memory_file,gpu_output_file):
    """Returns map of GPU id to memory allocated on that GPU."""

    output = run_command("nvidia-smi")
    #print("nvidia-smi",output)
    gpu_output = output[output.find("GPU Memory"):]
    #print("GPU Memory",gpu_output)
    save_txt(gpu_output,gpu_memory_file)
    save_txt(output,gpu_output_file)
    # lines of the form
    # |    0      8734    C   python                                       11705MiB |
    memory_regex = re.compile(r"[|]\s+?(?P<gpu_id>\d+)\D+?(?P<pid>\d+).+[ ](?P<gpu_memory>\d+)MiB")
    #print("memory_regex",memory_regex)
    rows = gpu_output.split("\n")
    #print("rows",rows)
    result = {gpu_id: 0 for gpu_id in list_available_gpus()}
    #print("result",result)
    for row in gpu_output.split("\n"):
        m = memory_regex.search(row)
        if not m:
            continue
        gpu_id = int(m.group("gpu_id"))
        gpu_memory = int(m.group("gpu_memory"))
        result[gpu_id] += gpu_memory
    return result

In [34]:
def save_txt(txt,file):
    text_file = open(file, "w")
    text_file.write(txt)
    text_file.close()

In [37]:

import os
#import GPUtil as GPU
import tensorflow as tf
#import threading
import timeit
import time
#from tensorflow import logging
from tensorflow.python.client import timeline
#os.environ['TF_CPP_MIN_VLOG_LEVEL']='3'
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
#logging.set_verbosity(logging.INFO)
#logging.vlog(1, "Registering %s (%s) in %s.", name, candidate, self._name)

config = tf.ConfigProto(device_count={"GPU": 2,"CPU":1},
                        allow_soft_placement=True,
                        inter_op_parallelism_threads=16,
                        intra_op_parallelism_threads=16,
                        use_per_session_threads=True,
                        log_device_placement=True)
config.gpu_options.allow_growth = True

#run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
#run_metadata = tf.RunMetadata()
print("************Start ALL*************")
start_gpu_memory=gpu_memory_map("gpu_memory_start.txt","gpu_out_start.txt")

save_txt(str(start_gpu_memory),"start_gpu_memory.txt")
#print(type(start_gpu_memory))
print("gpu_memory_map",start_gpu_memory)
#GPU.showUtilization(all=True)
#print("Start GPU",run_command("nvidia-smi -a"))
#import setGPU
#print("setGPU.stats",setGPU.stats)
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    GPU_LIST=[]
    list_file_gpu=[]
    import os
    #os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"
    with tf.device('/cpu'):
       
        #print("setGPU.bestGPU",setGPU.bestGPU)
        #print("setGPU.ratios",setGPU.ratios)
        #print("setGPU.ids",setGPU.ids)
        CPU_Start_Time=time.time()
        CPU_Start_Clock=time.clock()
        print("Start Time CPU",CPU_Start_Time, CPU_Start_Clock)
        start = timeit.default_timer()
        GPU_LIST=get_available_gpus()
        print("GPU_LIST",GPU_LIST)
        if len(GPU_LIST) > 1:
            list_file_gpu=chunkIt(a,len(GPU_LIST))
            for i in range(len(GPU_LIST)):
                start_time=time.time()
                start_oclock=time.clock()
                print("Start Time GPU",i,"Time is ",start_time,start_oclock)
                with tf.device(GPU_LIST[i]):

                    #gpu_full_process(file_names,path_data_base,path_tf,path_non_redundant,path_idf,file_path_sim)
                    #Your statements here
                    print("In GPU Dev",i)
                    model(list_file_gpu[i],i)
                end_time=time.time()
                end_oclock=time.clock()
                print("End Time GPU",i,"Time is ",end_time,end_oclock)
                total_time_GPU=end_time-start_time
                total_clock_GPU=end_oclock-start_oclock
                print("total_time_GPU",total_time_GPU)
                print("total_clock_GPU",total_clock_GPU)
            
        else:
            list_file_gpu=a
            start_time=time.time()
            start_oclock=time.clock()
            print("Start Time GPU",0,"Time is ",start_time,start_oclock)
            with tf.device(GPU_LIST[0]):
                
                #gpu_full_process(file_names,path_data_base,path_tf,path_non_redundant,path_idf,file_path_sim)
                #Your statements here
                print("In GPU Dev",0)
                model(list_file_gpu,0)
            end_time=time.time()
            end_oclock=time.clock()
            print("End Time GPU",0,"Time is ",end_time,end_oclock)
            total_time_GPU=end_time-start_time
            total_clock_GPU=end_oclock-start_oclock
            print("total_time_GPU",total_time_GPU)
            print("total_clock_GPU",total_clock_GPU)
        #list_file_gpu=chunkIt(a,2)
        #print(list_file_gpu)
        
        
        print("************End GPU*************")
        #print("setGPU.stats",setGPU.stats)
        #print("setGPU.bestGPU",setGPU.bestGPU)
        #print("setGPU.ratios",setGPU.ratios)
        #print("setGPU.ids",setGPU.ids)
        #print("gpu_memory_map",gpu_memory_map())
        CPU_End_Time=time.time()
        CPU_End_Clock=time.clock()
        stop = timeit.default_timer()
        print("CPU End Time",CPU_End_Time, CPU_End_Clock)
        #GPU.showUtilization(all=True)#https://github.com/anderskm/gputil
        End_gpu_memory=gpu_memory_map("gpu_memory_end.txt","gpu_out_end.txt")

        save_txt(str(End_gpu_memory),"End_gpu_memory.txt")
        print("gpu_memory_map",End_gpu_memory)
        sess.close()
        #GPU.showUtilization(all=True)
        #print("End GPU",run_command("nvidia-smi -a"))
        #tl = timeline.Timeline(run_metadata.step_stats)
        #print("tl.generate_chrome_trace_format",tl.generate_chrome_trace_format(show_memory=True))

************Start ALL*************
gpu_memory_map {0: 0}
Start Time CPU 1512811945.765454 11.877256
GPU_LIST []
Start Time GPU 0 Time is  1512811945.766356 11.878584


IndexError: list index out of range

In [ ]:
tf.reset_default_graph()
End_gpu_memory_end_session=gpu_memory_map("gpu_memory_end_session.txt","gpu_out_end_session.txt")

save_txt(str(End_gpu_memory_end_session),"End_gpu_memory_end_session.txt")
